### Question 5. Three biggest pickup zones
Which were the top pickup locations with over 13,000 in total_amount (across all trips) for 2019-10-18?

Consider only lpep_pickup_datetime when filtering by date.

In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"
DB_PORT = "5433"
DB_NAME = "ny_taxi"

engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [3]:
query = '''
    WITH 
    locations AS (
        SELECT 
            "PULocationID", 
            SUM(total_amount) AS total_revenue, 
            COUNT(*) AS trip_count
        FROM green_taxi_data
        WHERE DATE(lpep_pickup_datetime) = '2019-10-18'
        GROUP BY "PULocationID"
        HAVING SUM(total_amount) > 13000
        ORDER BY total_revenue DESC
    )

    SELECT "LocationID", "Zone", l.total_revenue, l.trip_count FROM public.taxi_zone_lookup
    INNER JOIN locations l ON l."PULocationID" = public.taxi_zone_lookup."LocationID";
'''

# Execute the query and fetch results
try:
    with engine.connect() as connection:
        result = pd.read_sql(query, connection)
        print(result)
except Exception as e:
    print(f"Error: {e}")

   LocationID                 Zone  total_revenue  trip_count
0          74    East Harlem North       18686.68        1236
1          75    East Harlem South       16797.26        1101
2         166  Morningside Heights       13029.79         764
